# 1 Processing

In [1]:
import os
HOME = os.getcwd()
print(HOME)

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO


In [2]:
#clearml mac
# %env CLEARML_WEB_HOST=https://app.clear.ml
# %env CLEARML_API_HOST=https://api.clear.ml
# %env CLEARML_FILES_HOST=https://files.clear.ml
# %env CLEARML_API_ACCESS_KEY=QO1VBBX9J2S2VYILQTGI
# %env CLEARML_API_SECRET_KEY=ERuc1S6o5SirQGugvYXDFjH9b9aNi0u8S3rpALzXMa8YPSLDMW

# clearml win
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
%env CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=Z4YDBU13VPOFHBHF8667
env: CLEARML_API_SECRET_KEY=JMzvRXn76AT83WuFJfS0FBGCY8c5TccbH5XboTYztrWqwzDdyn


In [3]:
# Pip install method (recommended)

!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.45 🚀 Python-3.11.5 torch-2.1.1 CPU (Apple M1 Max)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 952.0/1858.2 GB disk)


In [4]:
# read config file
import yaml

def load_config(config_file):
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    return config

config = load_config('seg_yolo_config_4.yaml')

print(config)

# project
project = config['project']
task_name = config['task_name']
exist_ok = config['exist_ok']

# dataset
# data = config['data_mac']
data = config['data_win']
test_data = config['test_data']
imgsz = config['imgsz']

# model
model = config['model']
task = config['task']
mode = config['mode']
# device = config['device_mac']
device = config['device_win']

# Training parameters
epochs = config['epochs']
batch = config['batch']
lr0 = config['lr0']
cos_lr = config['cos_lr']
weight_decay = config['weight_decay']
dropout = config['dropout']
optimizer = config['optimizer']
momentum = config['momentum']
seed = config['seed']

# Output configuration
# save = config['save']
# save_period = config['save_period']
plots = config['plots']
test_output = config['test_output']


{'project': 'YOLO-v8-seg', 'task_name': 'YOLO-seg-test-4', 'exist_ok': False, 'data_mac': '/Users/holmes/Documents/UNI-Bamberg/Arbeiten/Datensatz/Selfmade/3_iphone_domini_seg/YOLO/Dominik-seg.v2_yolo.yolov8/data.yaml', 'data_win': '/mnt/data/backup_pan/Dataset/Seg-dominik-iphone/YOLO/Dominik-seg.v2_yolo.yolov8/data.yaml', 'test_data': '/path/to/test_data.txt', 'imgsz': 640, 'model': 'yolov8m-seg.pt', 'task': 'segment', 'mode': 'train', 'device_mac': 'mps', 'device_win': 0, 'epochs': 200, 'batch': 16, 'lr0': 0.001, 'cos_lr': True, 'weight_decay': 0, 'dropout': 0.0, 'optimizer': 'Adam', 'momentum': 0.9, 'seed': 42, 'save': False, 'save_period': -1, 'plots': True, 'test_output': '/path/to/test_output'}


In [ ]:
import pandas as pd
import math

Dataset_version = "v2"
train_amount = 1113
batch_size = batch
total_epoch =  epochs
base_lr = lr0

test_record_csv = "/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/Test_seg_yolo_record.csv"

# Save in csv
new_row = [
    task_name,
    Dataset_version,
    model,
    train_amount,
    batch_size,
    total_epoch,
    base_lr,
    "none"
]

df = pd.read_csv(test_record_csv, header=None)
df.loc[len(df)] = new_row

df.to_csv(test_record_csv, header=False, index=False)

In [ ]:
import clearml
from clearml import Task

#Clear ML Initialization
cl_task = Task.init(project_name=project,task_name=task_name)
logger = cl_task.get_logger()

# 2 Training

In [ ]:
from ultralytics import YOLO

ini_model = "/home/jpan/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/yolov8m-seg.pt"
model = YOLO(ini_model)

results = model.train(project=project, name=task_name, exist_ok=exist_ok, data=data, imgsz=imgsz, task=task, device=device, epochs=epochs, batch=batch, lr0=lr0, cos_lr=cos_lr, weight_decay=weight_decay, dropout=dropout, optimizer=optimizer, momentum=momentum, seed=seed, plots=plots)

cl_task.close()

# 3 testing

## 3.1 path

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.45 🚀 Python-3.11.5 torch-2.1.1 CPU (Apple M1 Max)
Setup complete ✅ (10 CPUs, 32.0 GB RAM, 863.5/1858.2 GB disk)


In [2]:
from ultralytics import YOLO
import cv2
import os

model_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/weights/best.pt"

image_folder = "/Users/holmes/Documents/UNI-Bamberg/Arbeiten/RAW-DATA/FARO_pano_pc/test"

save_path = "/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-4/tests/test_more/resized"



## 3.2 resize

In [31]:
# Resize images
resized_image_folder = os.path.join(image_folder, "resized")
os.makedirs(resized_image_folder, exist_ok=True)

for file in os.listdir(image_folder):
    if file.endswith(('.jpg', '.JPG', '.jpeg', '.png', '.bmp')):
        img_path = os.path.join(image_folder, file)
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (640, 640))
        resized_img_path = os.path.join(resized_image_folder, os.path.splitext(file)[0] + "_resize" + os.path.splitext(file)[1])
        cv2.imwrite(resized_img_path, img_resized)
image_folder = resized_image_folder


## 3.3 test

In [9]:
import os
import cv2
import random
import numpy as np
from ultralytics import YOLO

# Paths
model_path = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-2/weights/best.pt'
save_path = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-2/tests'
image_folder = '/Users/holmes/Documents/UNI-Bamberg/Arbeiten/Datensatz/Selfmade/3_iphone_domini_seg/YOLO/Dominik-seg.v2_yolo.yolov8/test/images'

# Load model
model = YOLO(model_path)

# Create save directory if it does not exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Get image files from folder
image_files = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.endswith(('.jpg', '.jpeg', '.JPG', '.png', '.bmp'))]

# Process each image
for img_path in image_files:
    # Run YOLO prediction
    results = model.predict(img_path)

    # Read the image using OpenCV
    img = cv2.imread(img_path)

    # Iterate over each detection result
    for result in results:
        # Get the boxes, masks, and class ids from the result
        boxes = result.boxes.cpu().numpy()
        masks = result.masks.data.cpu().numpy()  # Extract segmentation masks

        # Generate random colors for the classes
        colors = [random.choices(range(256), k=3) for _ in range(len(boxes))]

        # Draw bounding boxes, masks, and confidence on the image
        for i, box in enumerate(boxes):
            class_id = int(box.cls[0])  # Extract the class ID
            confidence = box.conf[0]  # Extract the confidence score
            if confidence > 0.4:  # Filter low-confidence detections
                xyxy = box.xyxy[0].astype(int)
                confidence_text = f"{confidence:.2f}"

                # Draw bounding box
                color = colors[i]
                cv2.rectangle(img, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
                cv2.putText(img, confidence_text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                # Draw mask overlay
                mask = masks[i]  # Extract the mask for this detection
                mask = mask.astype(np.uint8)  # Convert mask to uint8
                mask_color = np.array(color, dtype=np.uint8)  # Convert color to array
                colored_mask = np.zeros_like(img, dtype=np.uint8)

                # Apply the mask color where mask value is 1
                colored_mask[mask == 1] = mask_color

                # Overlay the mask onto the image with some transparency
                img = cv2.addWeighted(img, 1, colored_mask, 0.5, 0)

    # Save the image with drawn results
    unique_save_path = os.path.join(save_path, os.path.basename(img_path))
    cv2.imwrite(unique_save_path, img)
    print(f"Image saved: {unique_save_path}")



image 1/1 /Users/holmes/Documents/UNI-Bamberg/Arbeiten/Datensatz/Selfmade/3_iphone_domini_seg/YOLO/Dominik-seg.v2_yolo.yolov8/test/images/Model_11_0007_jpeg.rf.f9ee744b5e2a29b27f34b6ef8c434445.jpg: 640x640 10 side_beams, 449.0ms
Speed: 2.4ms preprocess, 449.0ms inference, 11.5ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-2/tests/Model_11_0007_jpeg.rf.f9ee744b5e2a29b27f34b6ef8c434445.jpg

image 1/1 /Users/holmes/Documents/UNI-Bamberg/Arbeiten/Datensatz/Selfmade/3_iphone_domini_seg/YOLO/Dominik-seg.v2_yolo.yolov8/test/images/Model_15_0045_jpeg.rf.ac193e7eb06b5ddc3cd51d3a075eb4ad.jpg: 640x640 1 main_beam, 12 side_beams, 388.8ms
Speed: 1.4ms preprocess, 388.8ms inference, 12.6ms postprocess per image at shape (1, 3, 640, 640)
Image saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Y

In [13]:
import os
import cv2
import random
import numpy as np
import json
from ultralytics import YOLO

# Paths (provided by you)
model_path = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-2/weights/best.pt'
save_path = '/Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/YOLO/YOLO-v8-seg/YOLO-seg-test-2/tests'
image_folder = '/Users/holmes/Documents/UNI-Bamberg/Arbeiten/Datensatz/Selfmade/3_iphone_domini_seg/YOLO/Dominik-seg.v2_yolo.yolov8/test/images'

# Load YOLO model
model = YOLO(model_path)

# Create save directory if it does not exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Function to convert binary mask to polygons (optional, use as needed)
def binary_mask_to_polygons(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    polygons = [contour.flatten().tolist() for contour in contours if len(contour) > 2]  # Extract only contours with sufficient points
    return polygons

# Process each image
for file_name in os.listdir(image_folder):
    file_path = os.path.join(image_folder, file_name)
    if file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        # Read and process the image
        im = cv2.imread(file_path)  # Read image in BGR format
        if im is None:
            continue
        im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for YOLO
        original_shape = im.shape[:2]  # Store original image size (height, width)

        # Run YOLO prediction
        results = model.predict(im_rgb)

        detection_results = []

        for result in results:
            # Extract bounding boxes, masks, classes, and confidences
            boxes = result.boxes.cpu().numpy()
            masks = result.masks.data.cpu().numpy() if result.masks is not None else None  # Check if masks exist

            for i, box in enumerate(boxes):
                class_id = int(box.cls[0])  # Class ID
                confidence = float(box.conf[0])  # Confidence score
                xyxy = box.xyxy[0].tolist()  # Bounding box (x1, y1, x2, y2)

                # Store the detection result
                instance_result = {
                    "class": class_id,
                    "box": xyxy,
                    "score": confidence,
                }

                # If masks are available, resize them to the original image size and apply
                if masks is not None:
                    mask = masks[i].astype(np.uint8)
                    resized_mask = cv2.resize(mask, (original_shape[1], original_shape[0]))  # Resize mask to match original image size
                    
                    # Convert binary mask to polygons
                    polygons = binary_mask_to_polygons(resized_mask)
                    instance_result["mask"] = [{"polygon": polygon} for polygon in polygons]

                detection_results.append(instance_result)

        # Save the result to JSON
        result_dict = {
            "file_name": file_name,
            "instances": detection_results
        }

        # Save JSON with same name as image
        json_output_file_path = os.path.join(save_path, f"{os.path.splitext(file_name)[0]}.json")
        with open(json_output_file_path, 'w') as json_file:
            json.dump(result_dict, json_file, indent=4)

        # Optionally: save the visualized image
        test_name = f"det_seg_{file_name}"
        output_image_path = os.path.join(save_path, test_name)

        # Draw bounding boxes
        colors = [random.choices(range(256), k=3) for _ in range(len(boxes))]

        for i, box in enumerate(boxes):
            xyxy = [int(x) for x in box.xyxy[0]]  # Bounding box coordinates
            color = colors[i]

            # Draw bounding box
            cv2.rectangle(im, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, 2)
            confidence_text = f"{box.conf[0]:.2f}"
            cv2.putText(im, confidence_text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Save the image with bounding boxes and masks
        cv2.imwrite(output_image_path, im)
        print(f"Image and JSON saved: {output_image_path} and {json_output_file_path}")



0: 384x640 1 main_beam, 274.8ms
Speed: 1.6ms preprocess, 274.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Image and JSON saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/more_test/det_seg_5_2_3840x2160.jpg and /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/more_test/5_2_3840x2160.json

0: 384x640 (no detections), 254.9ms
Speed: 1.8ms preprocess, 254.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Image and JSON saved: /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_Detectron2/r101_test_6_0618_server/more_test/det_seg_5_1_3840x2160.jpg and /Users/holmes/Documents/UNI-Bamberg/4.Semester_MA/Masterthesis/xAI_Masterthesis_Pan/Code/Segmentation/Detectron/202406_